In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

import requests
import json
from random import randint
import os
import time
import json
import pickle

In [15]:
%cd ..
%cd demo Mativ
all_anime_df = pd.read_csv(os.path.join("data", "tags_anime.csv"))
%cd ..
%cd demo Victor
all_anime_df

e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Mativ
e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Victor


In [18]:
detalles_anime = {}
for i in range(1, 16):
    path = os.path.join("data", f"detalles_anime_{i}.pickle")
    with open(path, "rb") as archivo:
        detalles_anime = detalles_anime | pickle.load(archivo)

In [16]:
detalles_anime

{1: {'detalles': {'id': 1,
   'title': 'Cowboy Bebop',
   'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/4/19644.jpg',
    'large': 'https://cdn.myanimelist.net/images/anime/4/19644l.jpg'},
   'alternative_titles': {'synonyms': [],
    'en': 'Cowboy Bebop',
    'ja': 'カウボーイビバップ'},
   'start_date': '1998-04-03',
   'end_date': '1999-04-24',
   'synopsis': "Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con